## Daily feature pipeline

In [ ]:
import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", module="IPython")

project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if project_root not in sys.path:
    sys.path.append(project_root)
# Set the environment variables from the file <root_dir>/.env
from backend.models import config
settings = config.HopsworksSettings(_env_file=f".env")

HopsworksSettings initialized!


In [18]:
import datetime
import requests
import pandas as pd
import hopsworks
from backend import util
import datetime
from pathlib import Path
import json
import re
import great_expectations as ge
import warnings
warnings.filterwarnings("ignore")

### Initialize project

In [19]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 


2025-11-14 22:08:53,264 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-14 22:08:53,267 INFO: Initializing external client
2025-11-14 22:08:53,268 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-14 22:08:54,487 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279162


Initialize parameters, parameters for papermill otherwise from .env.

In [ ]:
today = datetime.date.today()

csv_file=f"{project_root}/backend/data/rahu-air-quality.csv"

util.check_file_path(csv_file)

# taken from ~/.env. You can also replace settings.AQICN_API_KEY with the api key value as a string "...."
if settings.AQICN_API_KEY is None:
    print("You need to set AQICN_API_KEY either in this cell or in ~/.env")
    sys.exit(1)

AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value() 
country = locals().get("country", settings.AQICN_COUNTRY)
city = locals().get("city", settings.AQICN_CITY)
street = locals().get("street", settings.AQICN_STREET)
aqicn_url = locals().get("aqicn_url", settings.AQICN_URL)
csv_file = locals().get("csv_file", f"{project_root}/backend/data/rahu-air-quality.csv")
# latitude, longitude = util.get_city_coordinates("stockholm")
latitude, longitude = 59.33, 18.07
print(latitude, longitude)

print(f"Found AQICN_API_KEY: {AQICN_API_KEY}")

secrets = hopsworks.get_secrets_api()
# Replace any existing secret with the new value
secret = secrets.get_secret("AQICN_API_KEY")
if secret is not None:
    secret.delete()
    print("Replacing existing AQICN_API_KEY")

secrets.create_secret("AQICN_API_KEY", AQICN_API_KEY)

File successfully found at the path: /Users/jonaslorenz/Desktop/Code_KTH/ScalableML/AirQualityPrediction/backend/data/rahu-air-quality.csv
59.33 18.07
Found AQICN_API_KEY: 85a05e409148fc36fa8a5aef727bc004a02fa69c
Replacing existing AQICN_API_KEY
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


Secret('AQICN_API_KEY', 'PRIVATE')

Get back the feature groups

In [21]:
air_quality_fg = fs.get_feature_group(
    name='air_quality_with_all_lags',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

Get today's air quality data.

In [22]:
aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,4.0,sweden,stockholm,stockholm-st-eriksgatan-83,2025-11-14,https://api.waqi.info/feed/@9461


Get weather forecast data.

In [23]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 59.25°N 18.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-14,2.20,0.0,13.450708,285.524170,stockholm
1,2025-11-15,1.40,0.0,11.252519,277.352295,stockholm
2,2025-11-16,3.10,0.0,10.805998,240.018448,stockholm
3,2025-11-17,0.35,0.0,10.703569,289.653900,stockholm
4,2025-11-18,-0.35,0.0,8.217153,298.810699,stockholm
5,2025-11-19,-0.15,0.0,4.582052,44.999897,stockholm
6,2025-11-20,1.50,0.8,15.277749,55.560955,stockholm


Compute lag for the last three days and upload today's air quality data into the air quality feature group.

In [24]:
aq_all = air_quality_fg.read()
aq_all["date"] = pd.to_datetime(aq_all["date"]).dt.tz_localize(None)

aq_loc = aq_all[
    (aq_all["country"] == country) &
    (aq_all["city"] == city) &
    (aq_all["street"] == street)
].copy()

# Combine history + today's pm25
combined = pd.concat([aq_loc, aq_today_df], ignore_index=True)
combined = combined.sort_values("date").reset_index(drop=True)

# ---- ADD ROLLING FEATURES ----
group_cols = ["country", "city", "street"]


for lag in [2, 3]:
    combined[f"pm25_rolling_{lag}d"] = (
        combined
        .groupby(group_cols)["pm25"]
        .transform(lambda s: s.rolling(lag, min_periods=1).mean().shift(1))
    )

# Lags: 1, 2, 3 days
for lag in [1, 2, 3]:
    combined[f"pm25_lag_{lag}"] = (
        combined.groupby(group_cols)["pm25"].shift(lag)
    )

# ---- Extract today's row ----
aq_today_ready = combined[
    combined["date"] == today.strftime("%Y-%m-%d")
].copy()

# Ensure feature types
for col in [
    "pm25_rolling_3d", "pm25_rolling_2d",
    "pm25_lag_1", "pm25_lag_2", "pm25_lag_3"
]:
    aq_today_ready[col] = aq_today_ready[col].astype("float32")

# ---- Insert into feature group ----
if aq_today_ready.shape[0] == 1:
    air_quality_fg.insert(aq_today_ready)

aq_today_ready.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.19s) 
2025-11-14 22:09:15,926 INFO: 	6 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279162/fs/1265774/fg/1717611


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_with_all_lags_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279162/jobs/named/air_quality_with_all_lags_1_offline_fg_materialization/executions


,date,pm25,country,city,street,url,pm25_rolling_3d,pm25_rolling_2d,pm25_lag_1,pm25_lag_2,pm25_lag_3
2517,2025-11-14,4.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@9461,15.666667,13.0,8.0,18.0,21.0


Insert today's weather.

In [25]:
weather_fg.insert(daily_df, wait=True)

2025-11-14 22:09:35,054 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279162/fs/1265774/fg/1638074


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279162/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-14 22:09:52,423 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-14 22:09:58,908 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-14 22:11:55,783 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-14 22:11:55,937 INFO: Waiting for log aggregation to finish.
2025-11-14 22:12:04,572 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 735353
         }
       },
       "result": {
         "observed_value": 4.5820512771606445,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-14T09:09:35.000054Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type":